In [1]:
import asyncio
import logging
from datetime import datetime

import influxdb_client
import pandas as pd
from IPython.display import Markdown, display

from tastytrade.common.logging import setup_logging
from tastytrade.config import RedisConfigManager
from tastytrade.config.enumerations import Channels
from tastytrade.connections import Credentials, InfluxCredentials
from tastytrade.connections.sockets import DXLinkManager
from tastytrade.connections.subscription import RedisSubscriptionStore
from tastytrade.messaging.processors import (
    RedisEventProcessor,
    TelegrafHTTPEventProcessor,
)
from tastytrade.utils.time_series import forward_fill

# Show all rows in pandas DataFrames
pd.set_option("display.max_rows", 100)
pd.set_option("display.max_columns", None)
pd.set_option("display.width", None)
pd.set_option("display.max_colwidth", None)

logging.getLogger().handlers.clear()

TEST = True
ENV = "Live"
DURATION = 15

EDT = 5

start_time = datetime(2025, 1, 1)

setup_logging(
    level=logging.INFO,
    log_dir="../logs",
    filename_prefix=f"{'dev' if TEST else 'prod'}_tastytrade",
    console=True,
    file=True,
)

loop = asyncio.get_event_loop()
loop.set_debug(True)
logging.getLogger("asyncio").setLevel(logging.DEBUG)

2025-10-31 01:30:57 - INFO:root:62:Logging initialized - writing to ../logs/dev_tastytrade_20251031.log


# Service Connections

In [ ]:
config = RedisConfigManager(env_file="/workspace/.env")
config.initialize(force=True)

credentials = Credentials(config=config, env="Live")

dxlink = DXLinkManager(subscription_store=RedisSubscriptionStore())
await dxlink.open(credentials=credentials)

influx_user = InfluxCredentials(config=config)
influxdb = influxdb_client.InfluxDBClient(
    url=influx_user.url, token=influx_user.token, org=influx_user.org
)

# Safely attach processors only if the router is initialized
router = dxlink.router
if router is None:
    raise RuntimeError("DXLink router not initialized after open()")
handlers_dict = getattr(router, "handler", None)
if handlers_dict is None:
    raise RuntimeError("DXLink router.handler mapping not initialized")
for h in handlers_dict.values():
    h.add_processor(TelegrafHTTPEventProcessor())
    h.add_processor(RedisEventProcessor())

2025-10-31 01:30:59 - INFO:tastytrade.config.manager:174:Initialized 19 variables from .env file in Redis
2025-10-31 01:30:59 - DEBUG:asyncio:848:Get address info api.tastyworks.com:443, type=<SocketKind.SOCK_STREAM: 1>, flags=<AddressInfo.AI_ADDRCONFIG: 32>
2025-10-31 01:30:59 - DEBUG:asyncio:858:Getting address info api.tastyworks.com:443, type=<SocketKind.SOCK_STREAM: 1>, flags=<AddressInfo.AI_ADDRCONFIG: 32> took 29.315ms: [(<AddressFamily.AF_INET: 2>, <SocketKind.SOCK_STREAM: 1>, 6, '', ('170.76.246.65', 443))]
2025-10-31 01:30:59 - DEBUG:asyncio:535:<asyncio.sslproto.SSLProtocol object at 0x7a6b97fa2b50> starts SSL handshake
2025-10-31 01:30:59 - DEBUG:asyncio:594:<asyncio.sslproto.SSLProtocol object at 0x7a6b97fa2b50>: SSL handshake took 45.5 ms
2025-10-31 01:30:59 - DEBUG:asyncio:1121:<asyncio.TransportSocket fd=82, family=2, type=1, proto=6, laddr=('172.18.0.2', 49156), raddr=('170.76.246.65', 443)> connected to None:None: (<asyncio.sslproto._SSLProtocolTransport object at 0x7

2025-10-31 01:31:00 - INFO:tastytrade.messaging.handlers:225:SETUP
2025-10-31 01:31:00 - INFO:tastytrade.messaging.handlers:228:AUTH_STATE:UNAUTHORIZED
2025-10-31 01:31:00 - INFO:tastytrade.messaging.handlers:228:AUTH_STATE:AUTHORIZED
2025-10-31 01:31:00 - INFO:tastytrade.messaging.handlers:231:CHANNEL_OPENED:1
2025-10-31 01:31:00 - INFO:tastytrade.messaging.handlers:231:CHANNEL_OPENED:3
2025-10-31 01:31:00 - INFO:tastytrade.messaging.handlers:231:CHANNEL_OPENED:5
2025-10-31 01:31:00 - INFO:tastytrade.messaging.handlers:231:CHANNEL_OPENED:7
2025-10-31 01:31:00 - INFO:tastytrade.messaging.handlers:231:CHANNEL_OPENED:9
2025-10-31 01:31:00 - INFO:tastytrade.messaging.handlers:231:CHANNEL_OPENED:11
2025-10-31 01:31:00 - INFO:tastytrade.messaging.handlers:231:CHANNEL_OPENED:99
2025-10-31 01:31:00 - INFO:tastytrade.messaging.handlers:236:FEED_CONFIG:5:COMPACT
2025-10-31 01:31:00 - INFO:tastytrade.messaging.handlers:236:FEED_CONFIG:7:COMPACT
2025-10-31 01:31:00 - INFO:tastytrade.messaging.han

# Market Data Subscriptions

In [16]:
start_time = datetime(2025, 10, 25)

symbols = ["BTC/USD:CXTALP", "NVDA", "AAPL", "QQQ", "SPY", "SPX"]  # , "/ESH25:XCME"]
# symbols = ["BTC/USD:CXTALP"]
# symbols = ["NVDA"]
# symbols = ["QQQ"]
# symbols = ["AAPL"]
# symbols = ["SPY"]
# symbols = ["SPX"]

intervals = ["1d", "1h", "30m", "15m", "5m", "m"]

# ticker subscriptions
await dxlink.subscribe(symbols)

# candle subscriptions
for symbol in symbols:
    for interval in intervals:
        coroutine = dxlink.subscribe_to_candles(
            symbol=symbol,
            interval=interval,
            from_time=start_time,
        )
        await asyncio.wait_for(coroutine, timeout=10)

2025-10-31 01:36:05 - DEBUG:asyncio:1121:<asyncio.TransportSocket fd=113, family=2, type=1, proto=6, laddr=('172.18.0.2', 59770), raddr=('172.18.0.4', 6379)> connected to redis:6379: (<_SelectorSocketTransport fd=113 read=polling write=<idle, bufsize=0>>, <asyncio.streams.StreamReaderProtocol object at 0x7a6b9027b7d0>)
2025-10-31 01:36:05 - INFO:tastytrade.connections.sockets:236:Added subscription: BTC/USD:CXTALP
2025-10-31 01:36:05 - INFO:tastytrade.connections.sockets:236:Added subscription: NVDA
2025-10-31 01:36:05 - INFO:tastytrade.connections.sockets:236:Added subscription: AAPL
2025-10-31 01:36:05 - INFO:tastytrade.connections.sockets:236:Added subscription: QQQ
2025-10-31 01:36:05 - INFO:tastytrade.connections.sockets:236:Added subscription: SPY
2025-10-31 01:36:05 - INFO:tastytrade.connections.sockets:236:Added subscription: SPX
2025-10-31 01:36:05 - INFO:tastytrade.connections.sockets:236:Added subscription: BTC/USD:CXTALP{=d}
2025-10-31 01:36:05 - INFO:tastytrade.connections

TimeoutError: 

In [17]:
# forward fill
for symbol in symbols:
    for interval in intervals:
        event_symbol = f"{symbol}{{={interval}}}"
        logging.debug("Forward-filling %s", event_symbol)
        forward_fill(symbol=event_symbol, lookback_days=5)

# Check | Market Data feeds

In [6]:
symbol = "SPX{=m}"
symbol = "BTC/USD:CXTALP{=5m}"
# symbol = "/ESH25:XCME{=m}"

display(Markdown(f"**Candle Feed:** {symbol}"))
router = dxlink.router
if router is None or getattr(router, "handler", None) is None:
    raise RuntimeError("Router or handler map not initialized")
handlers_dict = router.handler
candle_handler = handlers_dict.get(Channels.Candle)
if candle_handler is None:
    raise KeyError("Candle channel handler not available")
feed_proc = candle_handler.processors.get("feed")
if feed_proc is None:
    raise KeyError("'feed' processor missing on Candle handler")
frames = getattr(feed_proc, "frames", {})
if symbol not in frames:
    raise KeyError(f"No frames for symbol {symbol}")
(frames[f"{symbol}"].tail(5).sort(by="time", descending=True))

**Candle Feed:** BTC/USD:CXTALP{=5m}

eventSymbol,time,eventFlags,index,sequence,count,open,high,low,close,volume,bidVolume,askVolume,openInterest,vwap,impVolatility
str,datetime[μs],i64,i64,i64,i64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
"""BTC/USD:CXTALP{=5m}""",2025-10-16 15:50:00,0,7561847411363020800,0,1773,109183.22,109335.58,109173.21,109239.48,null,null,null,0.0,null,null
"""BTC/USD:CXTALP{=5m}""",2025-10-16 15:45:00,0,7561846122872832000,0,4688,109765.78,109781.5,109041.41,109184.13,null,null,null,0.0,null,null
"""BTC/USD:CXTALP{=5m}""",2025-10-16 15:40:00,0,7561844834382643200,0,2878,110264.11,110272.18,109540.69,109765.78,null,null,null,0.0,null,null
"""BTC/USD:CXTALP{=5m}""",2025-10-16 15:35:00,0,7561843545892454400,0,989,110368.1,110372.67,110215.39,110264.6,null,null,null,0.0,null,null
"""BTC/USD:CXTALP{=5m}""",2025-10-16 15:30:00,0,7561842257402265600,0,1375,110575.81,110582.57,110217.69,110364.17,null,null,null,0.0,null,null


In [7]:
display(Markdown(f"**Quote Feed:** {symbol}"))
router = dxlink.router
if router is None or getattr(router, "handler", None) is None:
    raise RuntimeError("Router or handler map not initialized")
quote_handler = router.handler.get(Channels.Quote)
if quote_handler is None:
    raise KeyError("Quote channel handler not available")
feed_proc = quote_handler.processors.get("feed")
if feed_proc is None:
    raise KeyError("'feed' processor missing on Quote handler")
feed_proc.pl

**Quote Feed:** BTC/USD:CXTALP{=5m}

eventSymbol,bidPrice,askPrice,bidSize,askSize
str,f64,f64,f64,f64
"""BTC/USD:CXTALP""",109000.0,109474.13,0.03,0.04
"""NVDA""",182.23,182.24,241.0,196.0
"""AAPL""",248.23,248.29,25.0,150.0
"""SPX""",6672.33,6677.43,null,null
"""QQQ""",604.27,604.28,135.0,25.0
"""SPY""",665.3,665.31,230.0,130.0


In [ ]:
# TODOS

# [x] Widen the plot
# [x] Remove the scroller at the bottom
# [x] move the legent and remove Price (that is obvious)
# [x] Add MACD
# [x] Fix HULL - Align w/ candlesticks
# [x] ERROR if no study data found

# [ ] Add RSI
# [ ] Add Volume Profile (?? ... /ES, SPY, etc)
# [ ] Add velocity metric

In [ ]:
await dxlink.close()